Installing Requirements & necessary imports

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

Python 3.10.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Organize & Verify dataset

In [ ]:
!unzip dataset_mobilenet.zip -d /content/

Archive:  dataset_mobilenet.zip
   creating: /content/annotations-1/
   creating: /content/annotations-1/train/
  inflating: /content/annotations-1/train/labels.json  
   creating: /content/annotations-1/train/.ipynb_checkpoints/
   creating: /content/annotations-1/train/images/
  inflating: /content/annotations-1/train/images/flir_60-photo_jpg.rf.3df1efdb3d4d114729eeda51e8d658c3.jpg  
  inflating: /content/annotations-1/train/images/flir_50-photo_jpg.rf.30ef9500860a10c419b62b4b4eecd80a.jpg  
  inflating: /content/annotations-1/train/images/flir_790-photo_jpg.rf.75e88e3de5bc6d454a034d4d47064588.jpg  
  inflating: /content/annotations-1/train/images/flir_429-photo_jpg.rf.1c99964d13a30fb62db7cea55ccadfeb.jpg  
  inflating: /content/annotations-1/train/images/flir_436-photo_jpg.rf.24fd3a71d704bf98e8225cd7bb927a04.jpg  
  inflating: /content/annotations-1/train/images/flir_2279-photo_jpg.rf.cd6d87897775707a92d1cf54513d540c.jpg  
  inflating: /content/annotations-1/train/images/flir_768-pho

In [ ]:
train_dataset_path = "/content/annotations-1/train"
validation_dataset_path = "/content/annotations-1/valid"

In [ ]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

1: Person
2: Vehicle4W


In [ ]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

train_data size:  1777
validation_data size:  305


Finetune Pretrained MobilenetV2

In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(learning_rate=0.1, batch_size=32, epochs=25, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


55/55 [==============================] - 82s 588ms/step - total_loss: 7.6918 - cls_loss: 7.5187 - box_loss: 0.0022 - model_loss: 7.6290 - val_total_loss: 0.7599 - val_cls_loss: 0.6358 - val_box_loss: 0.0012 - val_model_loss: 0.6971
Epoch 2/25
55/55 [==============================] - 23s 411ms/step - total_loss: 0.5429 - cls_loss: 0.4005 - box_loss: 0.0016 - model_loss: 0.4800 - val_total_loss: 0.3228 - val_cls_loss: 0.2065 - val_box_loss: 0.0011 - val_model_loss: 0.2599
Epoch 3/25
55/55 [==============================] - 22s 393ms/step - total_loss: 0.3206 - cls_loss: 0.1969 - box_loss: 0.0012 - model_loss: 0.2577 - val_total_loss: 0.2504 - val_cls_loss: 0.1406 - val_box_loss: 9.3762e-04 - val_model_loss: 0.1875
Epoch 4/25
55/55 [==============================] - 22s 404ms/step - total_loss: 0.2528 - cls_loss: 0.1443 - box_loss: 9.1265e-04 - model_loss: 0.1900 - val_total_loss: 0.2324 - val_cls_loss: 0.1255 - val_box_loss: 8.8154e-04 - val_model_loss: 0.1695
Epoch 5/25
55/55 [=========

Test Data Results

In [ ]:
test_dataset_path = "/content/annotations-1/test"
test_data = object_detector.Dataset.from_coco_folder(test_dataset_path, cache_dir="/tmp/od_data/test")
print("test_data size: ", test_data.size)
loss, coco_metrics = model.evaluate(test_data, batch_size=4)
print(f"Test loss: {loss}")
print(f"Test coco metrics: {coco_metrics}")

test_data size:  331
83/83 [==============================] - 7s 61ms/step - total_loss: 0.2509 - cls_loss: 0.1487 - box_loss: 7.9623e-04 - model_loss: 0.1885


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.20s).
Accumulating evaluation results...
DONE (t=0.29s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.765
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.983
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.928
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.214
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.721
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.834
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.801
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.803
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.803
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.425
 Av

In [ ]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

In [ ]:
!zip -r dataset_mobilenet.zip annotations-1/

  adding: annotations-1/ (stored 0%)
  adding: annotations-1/train/ (stored 0%)
  adding: annotations-1/train/labels.json (deflated 83%)
  adding: annotations-1/train/.ipynb_checkpoints/ (stored 0%)
  adding: annotations-1/train/images/ (stored 0%)
  adding: annotations-1/train/images/flir_60-photo_jpg.rf.3df1efdb3d4d114729eeda51e8d658c3.jpg (deflated 0%)
  adding: annotations-1/train/images/flir_50-photo_jpg.rf.30ef9500860a10c419b62b4b4eecd80a.jpg (deflated 0%)
  adding: annotations-1/train/images/flir_790-photo_jpg.rf.75e88e3de5bc6d454a034d4d47064588.jpg (deflated 0%)
  adding: annotations-1/train/images/flir_429-photo_jpg.rf.1c99964d13a30fb62db7cea55ccadfeb.jpg (deflated 1%)
  adding: annotations-1/train/images/flir_436-photo_jpg.rf.24fd3a71d704bf98e8225cd7bb927a04.jpg (deflated 1%)
  adding: annotations-1/train/images/flir_2279-photo_jpg.rf.cd6d87897775707a92d1cf54513d540c.jpg (deflated 0%)
  adding: annotations-1/train/images/flir_768-photo_jpg.rf.5daf210fe34f6d962a0ace90cc424b26.

In [ ]:
from google.colab import files
files.download('dataset_mobilenet.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>